## Simulation Tests

Fixed conditions comprise of sampling time (1600 timesteps to reflect 20 minute flight in seconds), survey area (7500m x 7500m) and sensor detection zone (based on camera model). Changing variables per flight will reflect survey effort and animal availability:

1.Survey Effort

* UAV path (Lawn-mower/ Figure 8)
* FOV (BIOT/ Maldives)
* UAV speed (lowest speed-average/ average-max speed)

2.Animal availability

* Animal movement model (Straight Line/ Stop-Start/ Random Walk)
* Animal speed (1-average/ average to max?)
* Availability bias (None/Sharks/Rays/Birds)


### Parameter Functions
These functions include:
* Selecting detection zone
* Start coord of UAV depending on camera type
* Establishing seperate speed profiles
* Sample points - number of coordinates and x, y values between 2 points in flight path

In [6]:
def detection_zone(camera):
    # FOV calculated per camera model in Strip_Sampling_Density.
    
    camdict = {"garmin": [83,113], "sony": [113, 75]}
    try:
        return camdict[camera.lower()]
    except KeyError:
        raise KeyError ('Invalid camera model used, should be :{}'.format([x for x in camdict.keys()]))

In [30]:
detection_zone('GARMIN')

[41.5, 56.5]

In [32]:
def boundary_coords_uav(camera):
    # x and y coordinates 
    
    area = detection_zone(camera)
    half_area = [x/2  for x in area]
    boundary = {'coord_1' : [(0 + half_area[0]), (0 + half_area[1])],
                'coord_2' : [(7500 - half_area[0]), (0 + half_area[1])],
                'coord_3' : [(7500 - half_area[0]), (7500 - half_area[1])],
                'coord_4' : [(0 + half_area[0]), (7500 - half_area[1])]}
    
    return boundary

boundary_coords_uav('sony')

{'coord_1': [56.5, 37.5],
 'coord_2': [7443.5, 37.5],
 'coord_3': [7443.5, 7462.5],
 'coord_4': [56.5, 7462.5]}

In [23]:
from scipy.stats import truncnorm

def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
    # More intuitive method to use truncnorm for speed_profile distribution

    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)



def speed_profile (level, mean = 18, sd = 1, low = 4, upp = 33):
    # Speed profile for random sample distribution
    # Average speed: 18 m/s 
    # Level 1 = low, Level 2 = high
    
    speed_dist = get_truncated_normal(mean, sd, low, upp)
    value = speed_dist.rvs(1) # Select 1 random number from this distribution
    if level == 1:
        while value > mean:
            value = speed_dist.rvs(1)
    else:
        while value < mean:
            value = speed_dist.rvs(1)
    return value


In [25]:
speed_profile(2)

array([ 20.42296313])

In [84]:
import math

def find_sample_points(start_coord, end_coord, angle, speed_level):

    # Retrieves coordinates for x number of timesteps between 2 points

    horiz_dist = abs(end_coord[0] - start_coord[0]) # x coordinates
    vert_dist = abs(end_coord[1] - start_coord[1]) # y coordinates
    length = math.sqrt((horiz_dist**2) + (vert_dist**2)) # length in metres between both points
    angle_radians = math.radians(angle)

    total_x = 0 # Sum of all x values (should be below x value of end_coord)
    total_y = 0 # Sum of all y values (should be below x value of end_coord)
    total_length_covered = 0
    x_list = [start_coord[0]] # list of x values at every step
    y_list = [start_coord[1]] # list of y values at every step
    iteration = 0 # used to cumulatively add the last x, y values to recent

    while True:
        s = speed_profile(speed_level) # get our speed for this step
        print('speed is:', s)
        total_length_covered += s # cumulatively add to total length covered
        if total_length_covered > length:
            break

        delta_x = x_list[iteration] + (math.cos(angle_radians)* s) # From start x coordinate, add the length with direction calculated through cosine 
        delta_y = y_list[iteration] + (math.sin(angle_radians)* s) # From start y coordinate, add the length with direction calculated through sine
        print('x_list so far is:', x_list)
        print('value being added from x_list is:', x_list[iteration])
        print('delta_x is:', delta_x)
        #total_x += delta_x # add the new x value cumulatively
        #print('total_x is:', total_x)
        total_y += delta_y # add the new y value cumulatively
        x_list.append(x_list[iteration] + delta_x) # adding the new coordinates cumulatively???
        y_list.append(y_list[iteration] + delta_y)

        iteration += 1


    sample_coords = zip(x_list, y_list)     # zip up to create list of tuples, each being a set of coordinates where a sample has been taken
    
    #print(iteration)
    #print(total_x)
    #print(total_y)
    #print(total_length_covered)
    #print(sample_coords)
    
    
    return True

In [85]:
find_sample_points([0,0], [100,0], 360, 1)

#angle_radians = math.radians(360)
#delta_y = 0 + (math.sin(angle_radians)* 200)
#delta_y

speed is: [ 15.94586697]
x_list so far is: [0]
value being added from x_list is: 0
delta_x is: [ 15.94586697]
speed is: [ 17.4443456]
x_list so far is: [0, array([ 15.94586697])]
value being added from x_list is: [ 15.94586697]
delta_x is: [ 33.39021257]
speed is: [ 16.0588935]
x_list so far is: [0, array([ 15.94586697]), array([ 49.33607955])]
value being added from x_list is: [ 49.33607955]
delta_x is: [ 65.39497305]
speed is: [ 15.23934959]
x_list so far is: [0, array([ 15.94586697]), array([ 49.33607955]), array([ 114.7310526])]
value being added from x_list is: [ 114.7310526]
delta_x is: [ 129.97040219]
speed is: [ 17.96244666]
x_list so far is: [0, array([ 15.94586697]), array([ 49.33607955]), array([ 114.7310526]), array([ 244.70145478])]
value being added from x_list is: [ 244.70145478]
delta_x is: [ 262.66390144]
speed is: [ 16.10579935]
x_list so far is: [0, array([ 15.94586697]), array([ 49.33607955]), array([ 114.7310526]), array([ 244.70145478]), array([ 507.36535623])]
va

True